<a href="https://colab.research.google.com/github/ahmettalhasen/Text-Similarity/blob/master/projectSupportCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import re
from nltk.stem.porter import PorterStemmer
import time
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# Importing the dataset
df = pd.read_csv('gdrive/My Drive/summer2019/support_forum_questions.csv',sep="|")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#Replacing Nans in 'tags' with 
# df.loc[df.tags.isnull()]['tags'] = ' '

In [0]:
df.dropna(axis = 0, inplace = True)
df.isnull().sum()

df = df.sample(frac=1.0)
df = df.reset_index(drop=True)

df = df.drop('login', axis=1)
df = df.drop('added', axis=1)

In [0]:
import re

#Clearing Html function
def clearhtml(raw_html):
   cleanr = re.compile('<.*?>')
   cleantext = re.sub(cleanr, ' ', raw_html)
   return cleantext

#Clearing all the irrelevant things
def clearstuff(text):
  cleantext = re.sub('\n', ' ', text)
  cleantext = re.sub('\r', ' ', cleantext)
  cleantext = re.sub('&nbsp', ' ', cleantext)
  return cleantext

#def clearIRwords(text):
#  cleantext = re.sub('hello', ' ', text)
#  cleantext = re.sub('thanks', ' ', cleantext)
#  cleantext = re.sub('thank you', ' ', cleantext)
#  cleantext = re.sub('hi(\s)', ' ', cleantext)
#  cleantext = re.sub('aytekin', ' ', cleantext)
#  return cleantext

def initial_clean(text):
    """
    Function that cleans emails, websites and any symbols/punctuations
    """
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub("[^a-zA-Z ]", " ", text)
    text = text.lower() 
    return text

def preprocess(text):
  text = initial_clean(clearstuff(clearhtml(text)))
  return text


### Getting text tokenized and ready to process ###

def tokenizer(text):
    text = nltk.word_tokenize(text)
    return text  
  
stop_words = stopwords.words('english')
def remove_stop_words(text):
    """
    Function that removes all stopwords from text
    """
    return [word for word in text if word not in stop_words]

stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words, so plural and singular are treated the same
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    """
    This function applies all the functions above into one *
    """ 
    return stem_words(remove_stop_words(tokenizer(text)))
  

In [0]:
#Clearing html and updating data frame
t1 = time.time()
df['question'] = df['question'].apply(preprocess)
df['details'] = df['details'].apply(preprocess)
t2 = time.time()
print("Time to clean Html", len(df), "articles:", (t2-t1)/60, "min")

Time to clean Html 227934 articles: 27.781556793053944 min


In [0]:
df['quest'] = df['question'] + ' ' + df['details']

In [0]:
#Deleting the questions less tah. 30 character size -- They are all test entries or spams
df = df[df.quest.str.len() > 30 ]

In [0]:
#Copying the dataframe just in case
dfCopy = df.copy()

In [0]:
!pip install langdetect
from langdetect import detect 
from langdetect.lang_detect_exception import LangDetectException

def filter_language(text):
    try:
        return detect(text)
    except LangDetectException:
        pass

In [0]:
#Filtering the non-english questions out
dfEx = df[df.quest.apply(filter_language) == 'en']

In [0]:
df = dfEx
df = df.reset_index(drop=True)
df

In [0]:
#Clearing punctuation, tokenizing the words and dropping the stop_words
t3 = time.time()
df['tokenized'] = df['quest'].apply(apply_all)
t4 = time.time()
print("Time to clean and tokenize", len(df), "questions:", (t4-t3)/60, "min")

Time to clean and tokenize 220421 articles: 5.324121356010437 min


In [0]:
## Saving the latest verison as csv for practical use
export_csv = df.to_csv ('gdrive/My Drive/summer2019/cleanSupportData.csv', index = None, header=True)

In [0]:
d = pd.read_csv('gdrive/My Drive/summer2019/cleanSupportData.csv')
